In [2]:
# ================================================================
# 1. Pip install once per kernel  (≈ 1-2 min)
# ================================================================
!pip install -qU trl transformers datasets peft accelerate --extra-index-url https://download.pytorch.org/whl/cu118


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.1.1 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 1.9.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 1.1.1 requires transformers[sentencepiece]<4.41.0,>=4.38.0, but you have transformers 4.53.2 which is incompatible.
autogluon-timeseries 1.1.1 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 1.9.0 which is incompatible.
autogluon-timeseries 1.1.1 requires gluonts==0.15.1, but you have gluonts 0.14.3 which is incompatible.
autogluon-timeseries 1.1.1 require

In [15]:
# clean out any mismatching Accelerate first
!pip uninstall -y accelerate

# install a version in the allowed window, plus the other libs
!pip install -q "transformers==4.40.1" "accelerate==0.26.1" "trl==0.19.1" "peft" "datasets" --upgrade


ERROR: Cannot install accelerate==0.26.1 and trl==0.19.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [21]:
!pip uninstall -y accelerate trl transformers
#!pip install -q "transformers==4.49.0"
!pip install -q "transformers==4.39.3"


Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires accelerate<0.22.0,>=0.21.0, which is not installed.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-timeseries 1.1.1 requires accelerate<0.22.0,>=0.21.0, which is not installed.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-timeseries 1.1.1 requires gluonts==0.15.1, but you have gluonts 0.14.3 which is incompatible.


In [5]:
!pip uninstall -y accelerate
!pip uninstall -y autogluon-multimodal autogluon-timeseries gluonts nvidia-ml-py3
!pip install -U "accelerate>=0.27" "trl==0.19.1"

Found existing installation: autogluon.multimodal 1.1.1
Uninstalling autogluon.multimodal-1.1.1:
  Successfully uninstalled autogluon.multimodal-1.1.1
Found existing installation: autogluon.timeseries 1.1.1
Uninstalling autogluon.timeseries-1.1.1:
  Successfully uninstalled autogluon.timeseries-1.1.1
Found existing installation: gluonts 0.14.3
Uninstalling gluonts-0.14.3:
  Successfully uninstalled gluonts-0.14.3
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached trl-0.19.1-py3-none-any.whl.metadata (10 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
Using cached trl-0.19.1-py3-none-any.whl (376 kB)
Using cached accelerate-1.9.0-py3-none-any.whl (367 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalli

In [1]:
import transformers
print("transformers", transformers.__version__)



transformers 4.39.3


In [3]:

# ================================================================
# 0. House-keeping:  choose a role, bucket and paths
# ================================================================
import sagemaker, boto3, json, os, pathlib, time
sess = sagemaker.Session()
role = sagemaker.get_execution_role()                    # studio-execution-role
region = sess.boto_region_name
bucket = sess.default_bucket()                           # or hard-code yours
prefix = "llama3_math_rlhf"                              # keep all assets here

BASE_ID = "meta-textgeneration-llama-3-8b"    # JumpStart Neuron model
print(role, bucket, region)
s3_rm = f"s3://{bucket}/{prefix}/data/rm.jsonl"

arn:aws:iam::842747873802:role/service-role/AmazonSageMaker-ExecutionRole-20250624T170706 sagemaker-us-west-2-842747873802 us-west-2


Saved train_reward.py


In [5]:
from sagemaker.huggingface import HuggingFace
import time, sagemaker

role   = sagemaker.get_execution_role()
sess   = sagemaker.Session()

reward_est = HuggingFace(
    entry_point         = "train_reward.py",
    source_dir          = "./",          # has script + requirements.txt
    role                = role,
    instance_type       = "ml.g5.12xlarge",
    instance_count      = 1,
    transformers_version= "4.49.0",
    pytorch_version     = "2.5.1",
    py_version          = "py311",
    environment = {
        "HF_EULA_ACCEPT": "true",
        "HF_TOKEN"      : "",     # 🔑 your token
        "BASE_ID"       : "meta-llama/Meta-Llama-3-8B",  # or …-Instruct
        "TRAIN_FILE"    : s3_rm
    }
)

reward_est.fit(job_name=f"rm-{int(time.time())}")

rm_uri = reward_est.model_data
print("Reward model artefact:", rm_uri)


KeyboardInterrupt: 

In [2]:
!pip install -U sagemaker


  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sagemaker_core-1.0.46-py3-none-any.whl.metadata (4.9 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached platformdirs-4.3.8-py3-none-any.whl.metadata (12 kB)
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.7 MB/s eta 0:00:00
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached sagemaker_core-1.0.46-py3-none-any.whl (418 kB)
Using cached platformdirs-4.3.8-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 120.1 MB/s eta 0:00:00
Using cached mock-4.0.3-py3-none-any.whl (28 kB)
Using cached pydantic-2.11.7-py3-none-any.

In [14]:
from sagemaker.huggingface import HuggingFace
import time, sagemaker

role   = sagemaker.get_execution_role()
sess   = sagemaker.Session()

reward_est = HuggingFace(
    entry_point         = "train_reward.py",
    source_dir          = "./",          # has script + requirements.txt
    role                = role,
    instance_type       = "ml.g5.12xlarge",
    instance_count      = 1,
    transformers_version= "4.49.0",
    pytorch_version     = "2.5.1",
    py_version          = "py311",
    environment = {
        "HF_EULA_ACCEPT": "true",
        "HF_TOKEN"      : "",     # 🔑 your token
        "BASE_ID"       : "meta-llama/Meta-Llama-3-8B",  # or …-Instruct
        "TRAIN_FILE"    : s3_rm,
        "PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:True"
    }
)

reward_est.fit(job_name=f"rm-{int(time.time())}")

rm_uri = reward_est.model_data
print("Reward model artefact:", rm_uri)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: rm-1752957473


2025-07-19 20:39:06 Starting - Starting the training job...
2025-07-19 20:39:14 Pending - Training job waiting for capacity......
2025-07-19 20:40:13 Pending - Preparing the instances for training...
2025-07-19 20:40:57 Downloading - Downloading the training image........................
2025-07-19 20:45:04 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-07-19 20:45:42,876 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-19 20:45:42,913 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-19 20:45:42,922 sagemaker_pytorch_container.training INFO     Block unti

In [9]:
import boto3, pprint
sm = boto3.client("sagemaker")
tj = sm.describe_training_job(TrainingJobName="rm-1752812991")
print("EnableNetworkIsolation:", tj["EnableNetworkIsolation"])
print("VpcConfig:", tj.get("VpcConfig"))
print("Image URI:", tj["AlgorithmSpecification"]["TrainingImage"])


EnableNetworkIsolation: False
VpcConfig: None
Image URI: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:2.5.1-transformers4.49.0-gpu-py311-cu124-ubuntu22.04


In [20]:
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput
import time, sagemaker, boto3, os, pathlib

s3_prompts = f"s3://{bucket}/{prefix}/data/ppo_prompts.jsonl"
rm_uri     = f"s3://sagemaker-us-west-2-842747873802/rm-1752957473/output/model.tar.gz"

# reward-model artefact → input channel
reward_data = TrainingInput(
    rm_uri,
    content_type="application/x-tar",
    distribution="FullyReplicated",
)

ppo_est = HuggingFace(
    entry_point         = "train_ppo.py",
    source_dir          = "./",
    role                = role,
    instance_type       = "ml.g5.48xlarge",
    instance_count      = 1,
    transformers_version= "4.49.0",
    pytorch_version     = "2.5.1",
    py_version          = "py311",
    environment = {
        "HF_EULA_ACCEPT": "true",
        "HF_TOKEN"      : "",
        "BASE_ID"       : "meta-llama/Meta-Llama-3-8B",
        "PROMPT_FILE"   : s3_prompts,              # S3 prompts file
        # NO RM_DIR here — the script will use SM_CHANNEL_REWARD
        "PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:True",
    },
)

ppo_est.fit(
    inputs   = {"reward": reward_data},            # ← defines the channel
    job_name = f"ppo-{int(time.time())}",
)

ppo_uri = ppo_est.model_data
print("PPO artefact:", ppo_uri)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: ppo-1753210881


2025-07-22 19:02:37 Starting - Starting the training job
2025-07-22 19:02:37 Pending - Training job waiting for capacity............
2025-07-22 19:04:22 Pending - Preparing the instances for training.........
2025-07-22 19:06:08 Downloading - Downloading the training image........................
2025-07-22 19:09:50 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-07-22 19:10:41,093 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-22 19:10:41,159 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-22 19:10:41,169 sagemaker_pytorch_container.training INFO     B

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:34                                                                                   │
│                                                                                                  │
│   31 │   },                                                                                      │
│   32 )                                                                                           │
│   33                                                                                             │
│ ❱ 34 ppo_est.fit(                                                                                │
│   35 │   inputs   = {"reward": reward_data},            # ← defines the channel                  │
│   36 │   job_name = f"ppo-{int(time.time())}",                                                   │
│   37 )                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:168 in wrapper  │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:139 in wrapper  │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                      

ls: cannot access '/opt/ml/input/data/reward': No such file or directory


CompletedProcess(args=['ls', '-R', '/opt/ml/input/data/reward'], returncode=2)

In [1]:
!pip uninstall -y accelerate trl transformers
!pip install -q "transformers==4.49.0"

Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires accelerate<0.22.0,>=0.21.0, which is not installed.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-timeseries 1.1.1 requires accelerate<0.22.0,>=0.21.0, which is not installed.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.4.2 which is incompatible.
autogluon-multimodal 1.1.1 requires transformers[sentencepiece]<4.41.0,>=4.38.0

In [1]:
# ================================================================
# 0. House-keeping:  choose a role, bucket and paths
# ================================================================
import sagemaker, boto3, json, os, pathlib, time
sess = sagemaker.Session()
role = sagemaker.get_execution_role()                    # studio-execution-role
region = sess.boto_region_name
bucket = sess.default_bucket()                           # or hard-code yours
prefix = "llama3_math_rlhf"                              # keep all assets here

BASE_ID = "meta-textgeneration-llama-3-8b"    # JumpStart Neuron model
print(role, bucket, region)
s3_rm = f"s3://{bucket}/{prefix}/data/rm.jsonl"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::842747873802:role/service-role/AmazonSageMaker-ExecutionRole-20250624T170706 sagemaker-us-west-2-842747873802 us-west-2


In [8]:
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput
import time, sagemaker, boto3, os, pathlib

s3_prompts = f"s3://{bucket}/{prefix}/data/ppo_prompts.jsonl"
rm_uri     = f"s3://sagemaker-us-west-2-842747873802/rm-1752957473/output/model.tar.gz"

# reward-model artefact → input channel
reward_data = TrainingInput(
    rm_uri,
    content_type="application/x-tar",
    distribution="FullyReplicated",
)

ppo_est = HuggingFace(
    entry_point         = "train_ppo.py",
    source_dir          = "./",
    role                = role,
    instance_type       = "ml.g5.48xlarge",
    instance_count      = 1,
    transformers_version= "4.49.0",
    pytorch_version     = "2.5.1",
    py_version          = "py311",
    environment = {
        "HF_EULA_ACCEPT": "true",
        "HF_TOKEN"      : "",
        "BASE_ID"       : "meta-llama/Meta-Llama-3-8B",
        "PROMPT_FILE"   : s3_prompts,              # S3 prompts file
        # NO RM_DIR here — the script will use SM_CHANNEL_REWARD
        "PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:True",
    },
)

ppo_est.fit(
    inputs   = {"reward": reward_data},            # ← defines the channel
    job_name = f"ppo-{int(time.time())}",
)

ppo_uri = ppo_est.model_data
print("PPO artefact:", ppo_uri)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: ppo-1753231906


2025-07-23 00:53:06 Starting - Starting the training job
2025-07-23 00:53:06 Pending - Training job waiting for capacity............
2025-07-23 00:54:41 Pending - Preparing the instances for training......
2025-07-23 00:56:02 Downloading - Downloading input data...
2025-07-23 00:56:22 Downloading - Downloading the training image..................
2025-07-23 00:59:39 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-07-23 01:00:33,563 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-23 01:00:33,627 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-23 01:00:3

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:34                                                                                   │
│                                                                                                  │
│   31 │   },                                                                                      │
│   32 )                                                                                           │
│   33                                                                                             │
│ ❱ 34 ppo_est.fit(                                                                                │
│   35 │   inputs   = {"reward": reward_data},            # ← defines the channel                  │
│   36 │   job_name = f"ppo-{int(time.time())}",                                                   │
│   37 )                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:168 in wrapper  │
│                                                                                                  │
│   165 │   │   │   │   │   caught_ex = e                                                          │
│   166 │   │   │   │   finally:                                                                   │
│   167 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 168 │   │   │   │   │   │   raise caught_ex                                                    │
│   169 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   170 │   │   │   else:                                                                          │
│   171 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:139 in wrapper  │
│                                                                                                  │
│   136 │   │   │   │   start_timer = perf_counter()                                               │
│   137 │   │   │   │   try:                                                                       │
│   138 │   │   │   │   │   # Call the original function                                           │
│ ❱ 139 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   140 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   141 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   142 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                      